添加引用

In [2]:
import utils
import params
import cv2
import matplotlib.pyplot as plt
import pickle
import numpy as np


Using TensorFlow backend.


从Tesla数据集的视频中，得到训练图片。从数据集的车辆转向记录中读取转向数据，将转向数据和训练图片做一一对应，得到训练数据集。

In [ ]:
def shuffle_in_unison(a, b):
     assert len(a) == len(b)
     shuffled_a = np.empty(np.array(a).shape, dtype=np.array(a).dtype)
     shuffled_b = np.empty(np.array(b).shape, dtype=np.array(b).dtype)
     permutation = np.random.permutation(len(a))
     for old_index, new_index in enumerate(permutation):
         shuffled_a[new_index] = a[old_index]
         shuffled_b[new_index] = b[old_index]
     return shuffled_a, shuffled_b

#加载epoch1到epoch9的所有视频的图片到img_list中，加载epoch1到epoch9的所有转向数据到steer_list，
for epoch_id in range(1,2):    
    vid_path = utils.join_dir(params.data_dir, 'epoch{:0>2}_front.mkv').format(epoch_id)
    steer_path = utils.join_dir(params.data_dir, 'epoch{:0>2}_steering.csv').format(epoch_id)
    steer_data = utils.fetch_csv_data(steer_path)
    steer_list = steer_data['wheel']

    frame_count = utils.frame_count(vid_path)
    cap = cv2.VideoCapture(vid_path)
    img_list = []
    
    for frame_id in range(frame_count):
        ret, img = cap.read()       
        img_resized = utils.img_pre_process(img)
        img_list.append(img_resized)
        
        if frame_id == frame_count - 1 and epoch_id == 9:
            print("img pre shape {}".format(img.shape))
            print("img after shape {}".format(img_resized.shape))
            plt.imshow(img)
            plt.show()
            plt.imshow(img_resized)
            plt.show()
    
    #随机打乱数据

  #  indices = np.random.permutation(len(steer_list)) 
  #  print(indices)
   # steer_list = np.array(steer_list)[indices] 
   # img_list = np.array(img_list)[indices]  
    steer_list,img_list = shuffle_in_unison(steer_list,img_list)
    #取10%的训练数据做验证集
    train_data_count = int((len(steer_list) * 0.9))
    img_list_train = img_list[(img_list[:train_data_count])]
    img_list_val = img_list[(img_list[train_data_count:])] 
    steer_list_train = steer_list[(steer_list[:train_data_count])] 
    steer_list_val = steer_list[(steer_list[train_data_count:])]
    
    #把训练集和验证集存到本地
    train_add = 'epoch{:0>2}_preprocess_train.p'.format(epoch_id)
    val_add = 'epoch{:0>2}_preprocess_val.p'.format(epoch_id)
    pickle.dump((img_list_train, steer_list_train_all), open(train_add, 'wb'))
    pickle.dump((img_list_val, steer_list_val_all), open(val_add, 'wb'))
    print (len(img_list_train))
    print(len(img_list_val))



加载预存数据

In [ ]:
train_features, train_steer = pickle.load(open('preprocess_train.p', mode='rb'))

In [ ]:
def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (32, 32, 3)
    : return: Numpy array of normalize data
    """
    # TODO: Implement Function
    a = 0
    b = 1
    min = 0
    max = 255
    x[(x < 0)]= 0;
    x[(x > 255)]= 255;
    
    return a + ( ( (x - min)*(b - a) )/( max - min ) )